In [1]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace  by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [1]:
import torch
import torch.nn as nn
import json
from transformers import BertTokenizer, BertModel
from sklearn.neighbors import KNeighborsClassifier

In [3]:
import json
list_sent=[]
Y_train=[]
with open('train.json', 'r') as file:
    data = file.read()
    data = json.loads(data)
    for label in data.keys():
        for sentence in data[label]:
            list_sent.append(sentence)
            Y_train.append(label)
    


In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

In [3]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [33]:
X_train=[]
for sent in list_sent:
    encoded_input = tokenizer(sent, return_tensors='pt')
    output = model(**encoded_input)
    X_train.append(output[1][0].detach().numpy())

In [37]:
# Assuming you have X_train and Y_train variables defined
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, Y_train)


KNeighborsClassifier(n_neighbors=1)

In [42]:
phrase="The role of credit scores in lending decisions is significant."
encoded_phrase = tokenizer(phrase, return_tensors='pt')
phrase_embedding = model(**encoded_phrase)[1][0].detach().numpy()
predicted_label = knn.predict([phrase_embedding])
print(predicted_label)


['Finance']


Feature based BERT

In [9]:
from transformers import BertTokenizer, BertModel
from sklearn.neighbors import KNeighborsClassifier
import json
import csv

class BertClassifFeatures():
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained("bert-base-uncased")
        self.knn = KNeighborsClassifier(n_neighbors=1)
    def train(self,train_file):
        list_sent=[]
        Y_train=[]
        with open(train_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent.append(sentence)
                    Y_train.append(label)
        X_train=[]
        for sent in list_sent:
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_train.append(output[1][0].detach().numpy())
        self.knn.fit(X_train, Y_train)
    def predict(self,phrase):
        encoded_phrase = self.tokenizer(phrase, return_tensors='pt')
        phrase_embedding = self.model(**encoded_phrase)[1][0].detach().numpy()
        predicted_label = self.knn.predict([phrase_embedding])
        return predicted_label
    def create_csv(self,test_file,csv_file):
        csv_file = open(csv_file, 'w',newline='')
        writer = csv.writer(csv_file)
        writer.writerow(["ID","Label"])
        i=0
        with open(test_file, 'r') as file:
            for line in file:
                pred=self.predict(line)[0]
                writer.writerow([str(i), str(pred)])
                i+=1
        csv_file.close()

In [10]:
test_file='./data/test_shuffle.txt'
csv_file='./data/feature_based_bert2.csv'	

In [11]:
bert=BertClassifFeatures()
bert.train('./data/train.json')

In [12]:
bert.create_csv(test_file,csv_file)

In [10]:
bert.predict("The Supreme Court has ruled in favor of the plaintiff in the high-profile case.")[0]

'Politics'

Finetuned BERT

In [58]:
import torch
import torch.nn as nn
import json
from transformers import BertTokenizer, BertModel
import random

class BertClassifFinetuned():
    def __init__(self,train_file,validation_file):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained("bert-base-uncased")
        self.linear = torch.nn.Linear(768,12,bias=True)
        self.softmax = torch.nn.Softmax(dim=1)
        self.train_file=train_file
        list_sent=[]
        list_lab=[]
        with open(train_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent.append(sentence)
                    list_lab.append(label)
        self.list_lab=list_lab
        self.list_sent=list_sent
        Y_train=[]
        self.dic_lab={}
        for i in range(len(list_lab)):
            if list_lab[i] not in self.dic_lab:
                self.dic_lab[list_lab[i]]=len(self.dic_lab)
            Y_train.append(self.dic_lab[list_lab[i]])
        X_train=[]
        for sent in list_sent:
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_train.append(output[1][0].detach().numpy())
        self.X_train=X_train
        self.Y_train=Y_train
        self.sent_train=list_sent
        self.lab_train=list_lab

        list_sent_val=[]
        list_lab_val=[]
        with open(validation_file, 'r') as file:
            data = file.read()
            data = json.loads(data)
            for label in data.keys():
                for sentence in data[label]:
                    list_sent_val.append(sentence)
                    list_lab_val.append(label)
        self.lab_val=list_lab_val
        self.sent_val=list_sent_val
        Y_val=[]
        X_val=[]
        for i,sent in enumerate(list_sent_val):
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_val.append(output[1][0].detach().numpy())
            Y_val.append(self.dic_lab[list_lab_val[i]])
        self.X_val=X_val
        self.Y_val=Y_val
    def train(self,epochs=10000):
        X_train=torch.tensor(self.X_train)
        Y_train=torch.tensor(self.Y_train)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(self.linear.parameters(), lr=0.001)
        for epoch in range(epochs):
            optimizer.zero_grad()
            outputs = self.linear(X_train)
            outputs = self.softmax(outputs)
            loss = criterion(outputs, Y_train)
            loss.backward()
            optimizer.step()
            if epoch % 1000 == 0:
                print ('Epoch [{}/{}], Loss: {:.4f}, Val score: {:.4f}'.format(epoch+1, epochs, loss.item(), self.validation_score()))
    def data_augmentation(self):
        list_sent = self.sent_train
        new_sent=[]
        list_lab = self.lab_train
        for phrase in list_sent:
            words=phrase.split(" ")
            i=random.randint(0,len(words)-1)
            words[i]="[MASK]"
            phrase=" ".join(words)
            new_sent.append(phrase)
        X_train=[]
        for sent in list_sent:
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_train.append(output[1][0].detach().numpy())
        self.X_train = self.X_train + X_train
        self.Y_train = self.Y_train + self.Y_train

    def get_new_train(self,conf=0.99,test_file='./data/test_shuffle.txt'):
        list_sent=[]
        list_lab=[]
        data=[]
        with open(test_file, 'r') as file:
            for line in file:
                pred=bert.predict(line)
                data.append([pred[1],pred[0],line])
        for x in data:
            if x[0]>conf:
                list_sent.append(x[2])
                list_lab.append(x[1])
        X_train=[]
        Y_train=[]
        for i,sent in enumerate(list_sent):
            encoded_input = self.tokenizer(sent, return_tensors='pt')
            output = self.model(**encoded_input)
            X_train.append(output[1][0].detach().numpy())
            Y_train.append(self.dic_lab[list_lab[i]])
        self.X_train = self.X_train + X_train
        self.Y_train = self.Y_train + Y_train
        self.sent_train = self.sent_train + list_sent
        self.lab_train = self.lab_train + list_lab
    def save(self,path='model.pth'):
        torch.save(self.linear.state_dict(), path)
    def load(self,path='model.pth'):
        self.linear.load_state_dict(torch.load(path))
    def predict(self,phrase):
        encoded_phrase = self.tokenizer(phrase, return_tensors='pt')
        phrase_embedding = self.model(**encoded_phrase)[1][0].detach().numpy()
        phrase_embedding=torch.tensor(phrase_embedding)
        predicted_labels = self.linear(phrase_embedding)
        predicted_labels=self.softmax(predicted_labels.unsqueeze(0))
        predicted_label=predicted_labels.argmax().item()
        proba = max(predicted_labels.detach().numpy()[0])
        label_name = next(key for key, value in bert.dic_lab.items() if value == predicted_label)
        return label_name , proba
    def validation_score(self):
        X_val=torch.tensor(self.X_val)
        Y_val=torch.tensor(self.Y_val)
        outputs = self.linear(X_val)
        outputs = self.softmax(outputs)
        predicted = outputs.argmax(1)
        correct = (predicted == Y_val).sum().item()
        accuracy = correct / len(Y_val)
        return accuracy    
    def create_csv(self,test_file,csv_file):
        csv_file = open(csv_file, 'w',newline='')
        writer = csv.writer(csv_file)
        writer.writerow(["ID","Label"])
        i=0
        with open(test_file, 'r') as file:
            for line in file:
                pred=self.predict(line)[0]
                writer.writerow([str(i), str(pred)])
                i+=1
        csv_file.close()


In [60]:
bert=BertClassifFinetuned('./data/train.json','./data/validation.json')

In [61]:
bert.train(epochs=30000)

Epoch [1/30000], Loss: 2.4864, Val score: 0.0667
Epoch [1001/30000], Loss: 2.4821, Val score: 0.1000
Epoch [2001/30000], Loss: 2.4778, Val score: 0.0667
Epoch [3001/30000], Loss: 2.4721, Val score: 0.1000
Epoch [4001/30000], Loss: 2.4636, Val score: 0.1000
Epoch [5001/30000], Loss: 2.4520, Val score: 0.1167
Epoch [6001/30000], Loss: 2.4422, Val score: 0.1167
Epoch [7001/30000], Loss: 2.4344, Val score: 0.1167
Epoch [8001/30000], Loss: 2.4273, Val score: 0.1167
Epoch [9001/30000], Loss: 2.4203, Val score: 0.1167
Epoch [10001/30000], Loss: 2.4131, Val score: 0.1500
Epoch [11001/30000], Loss: 2.4056, Val score: 0.1500
Epoch [12001/30000], Loss: 2.3980, Val score: 0.1500
Epoch [13001/30000], Loss: 2.3906, Val score: 0.1333
Epoch [14001/30000], Loss: 2.3835, Val score: 0.1333
Epoch [15001/30000], Loss: 2.3767, Val score: 0.1333
Epoch [16001/30000], Loss: 2.3703, Val score: 0.1333
Epoch [17001/30000], Loss: 2.3641, Val score: 0.1333
Epoch [18001/30000], Loss: 2.3581, Val score: 0.1167
Epoch 

In [67]:
bert.get_new_train(conf=0.9)

In [68]:
bert.data_augmentation()

In [69]:
bert.train(epochs=30000)

ValueError: Expected input batch_size (180) to match target batch_size (216).

In [65]:
phrase="The Supreme Court has ruled in favor of the plaintiff in the high-profile case."

In [66]:
bert.predict(phrase)

('Politics', 0.90307194)

In [55]:
import random

In [56]:
words=phrase.split(" ")
i=random.randint(0,len(words)-1)
words[i]="[MASK]"
phrase=" ".join(words)

In [57]:
phrase

'The Supreme [MASK] has ruled in favor of the plaintiff in the high-profile case.'

In [50]:
bert.validation_score()

0.16666666666666666

In [3]:
bert.load("./data/model.pth")

In [59]:
bert.save()

In [45]:
i=0
data=[]
with open(test_file, 'r') as file:
            for line in file:
                pred=bert.predict(line)
                data.append([pred[1],pred[0],line,i])
                i+=1

In [5]:
test_file='./data/test_shuffle.txt'
csv_file='./data/finetuned_based_bert.csv'	

In [11]:
data.sort(reverse=True)

In [48]:
new_X_train= []
new_Y_train= []
for x in data:
    if x[0]>0.9:
        new_X_train.append(x[2])
        new_Y_train.append(x[1])

In [49]:
len(new_X_train)

56

In [12]:
data

[[0.9996119,
  'Politics',
  'The senator is facing criticism for her stance on the recent bill.\n',
  1004],
 [0.99943036,
  'Food',
  'The best way to find the best local restaurants is to ask the locals.\n',
  396],
 [0.9986046,
  'Food',
  'The best way to find the best local food tours is to do your research ahead of time.\n',
  65],
 [0.9984767,
  'Food',
  'The best way to find the best local shopping is to ask the locals.\n',
  343],
 [0.9974995,
  'Food',
  'The best way to experience a new culture is to try the local food.\n',
  856],
 [0.99643517,
  'Food',
  'The best way to find the best local tours is to ask the locals or do your research ahead of time.\n',
  322],
 [0.9962063,
  'Politics',
  'The mayor has been accused of failing to address the needs of the community.\n',
  1073],
 [0.9962063,
  'Politics',
  'The mayor has been accused of failing to address the needs of the community.\n',
  587],
 [0.9958574,
  'Food',
  'The best way to find the best local cooking cla

In [24]:
bert.create_csv(test_file,csv_file)

In [40]:
phrase="The robotics competition inspires students to pursue careers in STEM fields."

In [41]:
bert.predict(phrase)

'Education'

TF IDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
td = TfidfVectorizer(max_features = 4500)
import json

list_sent=[]
Y_train=[]
with open("train.json", 'r') as file:
    data = file.read()
    data = json.loads(data)
    for label in data.keys():
        for sentence in data[label]:
            list_sent.append(sentence)
            Y_train.append(label)
X_train=[]


In [9]:
list_sent

['The mayor announced a new initiative to improve public transportation.',
 'The senator is facing criticism for her stance on the recent bill.',
 'The upcoming election has sparked intense debates among the candidates.',
 'Regular exercise and a balanced diet are key to maintaining good health.',
 'The World Health Organization has issued new guidelines on COVID-19.',
 'A new study reveals the benefits of meditation for mental health.',
 'The stock market saw a significant drop following the announcement.',
 'Investing in real estate can be a profitable venture if done correctly.',
 "The company's profits have doubled since the launch of their new product.",
 'Visiting the Grand Canyon is a breathtaking experience.',
 'The tourism industry has been severely impacted by the pandemic.',
 'Backpacking through Europe is a popular choice for young travelers.',
 'The new restaurant in town offers a fusion of Italian and Japanese cuisine.',
 'Drinking eight glasses of water a day is essentia

In [10]:
X = td.fit_transform(list_sent).toarray()

In [1]:
X

NameError: name 'X' is not defined

In [2]:
X_train

NameError: name 'X_train' is not defined